# Create and test three hypothesis:

## Read database and clean it up

In [43]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import math

In [44]:
#read database
sheet_id = '1VK2ZQyVCNNjh4D-aQ4xyRjU8DrsLhQ9O02FnZ9S_nSU'
df = pd.read_csv(f"https://docs.google.com/spreadsheets/d/{sheet_id}/export?format=csv")

In [45]:
#clean database
#nan values
df = df.dropna(how="any", axis=0) 
#values equal to 0
df = df[(df != 0).all(1)]

In [46]:
df

,user_id,task_name,rounded_hours,is_billed,brokenruledescription,project_name,client_id,warningid,spent_date,client_name,project_id,hours,client_currency
0,3226380.0,Programming,3.22,True,Time entry bigger than 3 hours.,Squad Vulcan,9385332.0,9385332_26566194_3226380,2020-10-19,Emerald Publishing,26566194.0,3.22,USD
1,3226316.0,QA,3.25,True,Time entry bigger than 3 hours.,Nova: Epic 271 UX/UI Design Process,9357853.0,9357853_26012326_3226316,2021-02-16,Emerald Works Limited - Nova,26012326.0,3.25,USD
2,1852603.0,UX/UI: Research / Ideation,4.09,True,Time entry bigger than 3 hours.,Theme Development,10259141.0,10259141_27179798_1852603,2021-05-03,Anderson International Ltd,27179798.0,4.09,GBP
3,1852603.0,UX/UI: High-end Fidelity Design,3.62,True,Time entry bigger than 3 hours.,MVP,10220982.0,10220982_27067840_1852603,2021-03-10,Resilience Engine,27067840.0,3.62,GBP
4,1852573.0,DevOps,13.92,True,Time entry bigger than 3 hours.,MVP,10220982.0,10220982_27067840_1852573,2021-05-03,Resilience Engine,27067840.0,13.92,GBP
...,...,...,...,...,...,...,...,...,...,...,...,...,...
22169,3898458.0,Dev: Pair Programming,2.53,True,Time entry without any notes.,Phase 3,10220982.0,10220982_30828522_3898458,2022-01-21,Resilience Engine,30828522.0,2.53,GBP
22172,3997286.0,Dev: Testing,3.32,True,Time entry without any notes.,OMS,9586049.0,9586049_30495793_3997286,2022-01-04,Gamifica,30495793.0,3.32,BRL
22174,2761130.0,UX/UI: High-end Fidelity Design,5.13,True,Time entry bigger than 3 hours.,Phase 3,10220982.0,10220982_30828522_2761130,2021-12-21,Resilience Engine,30828522.0,5.13,GBP
22194,1852602.0,UX/UI: High-end Fidelity Design,3.25,True,Time entry bigger than 3 hours.,User Onboarding Flow,9357853.0,9357853_30403711_1852602,2021-10-13,Emerald Works Limited - Nova,30403711.0,3.25,USD


# Hypothesys one: People who enter programming tasks enter more hours.

## Create dataframe

In [47]:
#create new df with important columns 
dftask51 = df[['user_id', 'task_name', 'hours']].copy()

In [48]:
dftask51 = dftask51.groupby(['user_id', 'task_name']).agg({'hours':'sum'}).reset_index()

## Classify user by programmers and not programmers

In [49]:
#fill user type column with "programmer" if task name is Programming or Dev: Pair Programming
dftask51.loc[(dftask51['task_name'] == 'Dev: Pair Programming') | (dftask51['task_name'] == 'Programming'), 'user_type'] = 'programmer'  
dftask51.loc[(dftask51['task_name'] != 'Dev: Pair Programming') & (dftask51['task_name'] != 'Programming'), 'user_type'] = 'not programmer'  

In [50]:
#sort users
dftask51 = dftask51.sort_values(["user_id","user_type"], ascending=[True, False])

In [51]:
# Classify users who have entered at least one programming task name as Programmer and those have not as not programmers
user_temp = ''

for i in dftask51.index: 
    if(dftask51["user_id"][i] == user_temp):
        tipoUser = dftask51["user_type"][i]
        dftask51["user_type"][i] = tipoUserTemp

    user_temp = dftask51["user_id"][i]
    tipoUserTemp = dftask51["user_type"][i]

<ipython-input-51-3d0565890449>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dftask51["user_type"][i] = tipoUserTemp


In [52]:
dftask51g = dftask51.groupby(['user_id','user_type']).agg({'hours':'mean'}).reset_index()

## create df for programmers

In [53]:
dfprogrammer = dftask51[dftask51['user_type'] == 'programmer']

## Run t test

In [54]:
#Run the T-test for the mean of ONE group of scores. See at: https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_1samp.html
stats.ttest_1samp(a = dfprogrammer['hours'], popmean = dftask51g['hours'].mean())

Ttest_1sampResult(statistic=-1.3467246425885888, pvalue=0.1791899767959702)

# Hypothesys two: People work more hours when the client's currency is GBP.

In [55]:
#create new df with important columns 
dftask52 = df[['user_id', 'client_currency', 'hours']].copy()

In [56]:
dftask52 = dftask52.groupby(['user_id', 'client_currency']).agg({'hours':'sum'}).reset_index()

## Create dataframe for GBP

In [57]:
#create dataframe for GBP entered hours
dfgbp = dftask52[dftask52['client_currency'] == 'GBP']

## Run t test

In [58]:
#Run the T-test for the mean of ONE group of scores. See at: https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_1samp.html
stats.ttest_1samp(a = dfgbp['hours'], popmean = dftask52['hours'].mean())

Ttest_1sampResult(statistic=-2.2951435718436737, pvalue=0.030391437008328866)

# Hypothesys three: People work less hours in december.

In [59]:
#create new df with important columns 
dftask53 = df[['user_id', 'spent_date', 'hours']].copy()

In [60]:
#get month from date_spent
dftask53['month'] = pd.to_datetime(df['spent_date']).dt.month
dftask53 = dftask53.drop(['spent_date'], axis = 1)

## Create df for december hours

In [61]:
#create df for only for december
dfdec = dftask53[dftask53['month'] == 12]

## Run t test

In [62]:
#Run the T-test for the mean of ONE group of scores. See at: https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_1samp.html
stats.ttest_1samp(a = dfdec['hours'], popmean = dftask53['hours'].mean())

Ttest_1sampResult(statistic=-4.97789577060998, pvalue=7.525432670171487e-07)

## Run correlation test

In [63]:
dftask53['month'].corr(dftask53['hours'])

0.022521589871756283